In [33]:
import os
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import metrics,preprocessing
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV

import scipy
path = os.getcwd()
path='/Users/xtarx/Documents/TUM/4th/MLMA/ex1/SAheartdata/'

In [168]:
data = pd.DataFrame.from_csv(path + "SAheart.csv",index_col=None)
data.famhist =pd.Categorical(data.famhist).codes
train_x = data.drop(['chd'], axis=1)
train_y = data['chd']

features = ['sbp','tobacco','ldl','adiposity','famhist','typea','obesity','alcohol','age']



In [130]:
data.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1


a) Create a model that contains only the intercept (null model), i.e. no features
are considered.

In [132]:
def null_model(y):
    logistic = LogisticRegression(C=1e6, solver='newton-cg', intercept_scaling=1e3)
    X = np.zeros(y.size).reshape(-1, 1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll,logistic.coef_

Create multiple models each considering a single feature. Note that famhist
is a categorical feature which has to be converted to numbers first.

In [140]:
def full_model(X, y):
    logistic = LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
    # X=X.reshape(-1,1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll,logistic.coef_

def model_wth_stats(X, y):
    logistic = LogisticRegression(C=1e6,solver='newton-cg',intercept_scaling=1e3)
    X=X.values.reshape(-1,1)
    logistic.fit(X, y)
    pred = logistic.predict(X)
    ll = metrics.log_loss(y, pred)
    return pred, ll,logistic.coef_

Create a function likelihood_ratio_test implementing the likelihood-ratio
test which takes the log-likelihood of the full model and the reduced model
(Section 1.1). Use this function to compare the single feature models to the
null model. Which feature yields the most significant improvement over the
null model? Make sure you consider the p-value of the likelihood-ratio test.

In [141]:
def likelihood_ratio_test(full_model_ll, reduced_model_ll, df):
    D = -2 * (full_model_ll - reduced_model_ll)
    p = scipy.special.gammaincc(df / 2, D / 2)
    return p

In [142]:
null_pred, null_ll,null_coef = null_model(train_y)
full_pred, full_ll,full_coef = full_model(train_x, train_y)

In [145]:
print("Model for feature:", features[4])
sbp_pred, sbp_ll,sbp_coef = model_wth_stats(train_x['famhist'], train_y)

Model for feature: famhist


In [146]:
sbp_ll

11.961481002566471

In [149]:
models = []
models_ll = []
p_value = []

#Null model


for i in range(0,len(features)):
    pred,ll,_=model_wth_stats(train_x[features[i]],train_y)
    models.append(pred)
    models_ll.append(ll)
    p_value.append(likelihood_ratio_test(full_ll, ll, 1))

In [150]:
features

['sbp',
 'tobacco',
 'ldl',
 'adiposity',
 'famhist',
 'typea',
 'obesity',
 'alcohol',
 'age']

In [151]:
p_value

[0.031326880803922312,
 0.13424775557170157,
 0.031326122482093953,
 0.048649733662181179,
 0.018671475844365633,
 0.018671475844365633,
 0.018671401970238567,
 0.017144204216065061,
 0.053187824578213541]

In [152]:
models_ll

[11.512947964465017,
 10.316810248590579,
 11.512968733229439,
 11.139167259708127,
 11.961481002566471,
 11.961481002566471,
 11.961484464027206,
 12.036243720293246,
 11.064472040465718]

d) Create a model which considers multiple features by starting with the null
model and adding one additional feature at a time. To determine which feature
to add, use the p-value as returned by the likelihood-ratio test. Extended
models with one additional feature, where the p-value is greater than 0.05,
should not be considered. In each step choose the model with the smallest
p-value. Continue until all features have been selected or the model cannot be
improved significantly any more. Print all selected features.

In [177]:
selected_features = []
min_p_value =0;
p_values_m = []
all_featues=list(features)

while min_p_value<0.05:
    p_values_m = []
    for i in range(0,len(all_featues)):
        temp_features = list(selected_features)
        temp_features.append(all_featues[i])
        _,new_ll,_ = full_model(train_x[temp_features],train_y)
        p_values_m.append(likelihood_ratio_test(full_ll,new_ll,1))
    min_p_value = min(p_values_m)
    if(min_p_value<0.05):
        selected_features.append(all_featues[p_values_m.index(min_p_value)])
        del all_featues[p_values_m.index(min_p_value)]
    
print (selected_features)


['alcohol', 'typea', 'obesity', 'adiposity']


In [180]:

_, ll_new,coef = full_model(train_x[selected_features], train_y)
print("ll new is ", ll_new)
coef

ll new is  11.7372759244


array([[ 0.00274485,  0.03382886, -0.13110714,  0.1317592 ]])

L1 (lasso) regularization can also be used for feature selection. Consider a
full model with all features as input, penalized with the L1 norm of coefficients
(try regularization parameter C in the range of 0.01 − 0.1). Features
with an non-zero coefficient are important for the classification. Compare the
Lasso-selected features to the features selected by p-values. Please note Lassofeature
selection requires a standardization of features that each feature has a
zero mean and a unit standard derivation (e.g.using Sklearn built-in function
sklearn.preprocessing.scale)

In [154]:
def lasso(X, y):
    X = preprocessing.scale(X)
    clf= LogisticRegression(penalty='l1', C = 0.1)
    clf.fit(X , y)
    return clf.coef_

In [155]:
lasso_coef=lasso(train_x, train_y)

In [156]:
lasso_coef

array([[ 0.03437361,  0.27791927,  0.23888362,  0.        ,  0.33218618,
         0.19224072,  0.        ,  0.        ,  0.54088522]])

In [157]:
print(features)

['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age']


In [158]:
print("Lasso selected features: ")

lasso_selected_features=[]
lasso_coef_list=lasso_coef.ravel()
for i in range(0, len(lasso_coef_list)):
    if(lasso_coef_list[i]!=0):
        lasso_selected_features.append(features[i])
    

Lasso selected features: 


In [159]:
lasso_selected_features

['sbp', 'tobacco', 'ldl', 'famhist', 'typea', 'age']